In [1]:
import pandas as pd
import numpy as np
#from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.utils import np_utils
from keras.models import Sequential
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
#import seaborn as sns
import openpyxl
from tensorflow.keras.layers import *
#from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from keras.layers import Dropout

In [2]:
data=pd.read_excel('hourly_data.xlsx',header=0)

In [3]:
def timeseries_to_supervised(df, n_in, n_out):
   agg = pd.DataFrame()

   for i in range(n_in, 0, -1):
      df_shifted = df.shift(i).copy()
      df_shifted.rename(columns=lambda x: ('%s(t-%d)' % (x, i)), inplace=True)
      agg = pd.concat([agg, df_shifted], axis=1)

   for i in range(0, n_out):
      df_shifted = df.shift(-i).copy()
      if i == 0:
         df_shifted.rename(columns=lambda x: ('%s(t)' % (x)), inplace=True)
      else:
         df_shifted.rename(columns=lambda x: ('%s(t+%d)' % (x, i)), inplace=True)
      agg = pd.concat([agg, df_shifted], axis=1)
   agg.dropna(inplace=True)
   return agg

In [4]:
only_atm_temperature=data['Atm.Temperature'].round(decimals = 2).to_frame(name='Atm.Temperature')

In [5]:
df= only_atm_temperature.copy()
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled  = min_max_scaler.fit_transform(np.reshape(df['Atm.Temperature'].values, (df.shape[0], 1)))
df_scaled  = pd.DataFrame(data=df_scaled , columns=['Atm.Temperature'])
n_in = 15 #Timeseries consists of 90 input steps.
n_out = 1
sdf = timeseries_to_supervised(df_scaled, n_in, n_out)
X, y = sdf[[('Atm.Temperature(t-%d)' % i) for i in range(15, 0, -1)]].values, sdf['Atm.Temperature(t)'].values
len_data = X.shape[0]
print("Data size: %d" % len_data)
train_size = int(len_data * .7)# 70% for training and 30% for testing
print ("Train  data size: %d" % train_size)
print ("Test data size: %d" % (len_data - train_size))

xtr, ytr = X[:train_size, :], y[:train_size]
xte, yte = X[train_size:, :], y[train_size:]
test_dates=data[train_size:].iloc[15:].Date #Keep the dates of the test dataset for display purposes
print(xtr.shape, ytr.shape)
print(xte.shape, yte.shape)

Data size: 8650
Train  data size: 6055
Test data size: 2595
(6055, 15) (6055,)
(2595, 15) (2595,)


In [6]:
batch_size = 1 #1
model0 = Sequential()

model0.add(Dense(units=50, input_dim=xtr.shape[1], activation="relu"))#input dim is 4 as the timeseries steps.
model0.add(Dense(50, activation="relu"))# linear activation functios was used as well
model0.add(Dropout(0.2))
model0.add(Dense(1))
model0.compile(loss='mean_squared_error', optimizer='adam')
model0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                800       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 3,401
Trainable params: 3,401
Non-trainable params: 0
_________________________________________________________________


In [7]:
model0.fit(xtr,ytr, epochs=10, batch_size=batch_size, verbose=2)

Epoch 1/10
6055/6055 - 2s - loss: 0.0059
Epoch 2/10
6055/6055 - 2s - loss: 0.0023
Epoch 3/10
6055/6055 - 2s - loss: 0.0019
Epoch 4/10
6055/6055 - 2s - loss: 0.0018
Epoch 5/10
6055/6055 - 2s - loss: 0.0017
Epoch 6/10
6055/6055 - 2s - loss: 0.0017
Epoch 7/10
6055/6055 - 2s - loss: 0.0017
Epoch 8/10
6055/6055 - 2s - loss: 0.0017
Epoch 9/10
6055/6055 - 2s - loss: 0.0016
Epoch 10/10
6055/6055 - 3s - loss: 0.0016


In [8]:
testPredict = model0.predict(xte, batch_size=batch_size)

In [9]:

yte2d = np.reshape(yte, (yte.shape[0], 1)) 
testPredict = min_max_scaler.inverse_transform(testPredict)
testY = min_max_scaler.inverse_transform(yte2d)
print("Test MSE: ", mean_squared_error(testY, testPredict))
print("Test MAE: ", sum(abs(testY-testPredict))/testY.shape[0])
print("Test R2: ", r2_score(testY, testPredict))
forecast=df_scaled[-15:]
forecast=forecast.to_numpy()
forecast=forecast.reshape(1,15)
n_past = 15
n_days_for_testing_prediction=16  #predict the next hour
predict_period_dates = pd.date_range(list(test_dates)[-n_past], periods=n_days_for_testing_prediction,freq='h').tolist()
predict_period_dates=predict_period_dates[-1:]


Test MSE:  1.1526993068816531
Test MAE:  [0.75012925]
Test R2:  0.9338360865256815


In [10]:
forecast = model0.predict(forecast, batch_size=batch_size)#predict

In [11]:
forecast = min_max_scaler.inverse_transform(forecast)#inverse
forecast=np.squeeze(forecast)#reshape
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i)

In [12]:
df_forecast_atm_temperature = pd.DataFrame({'Date':np.array(forecast_dates), 'Atm.Temperature Forecasted':forecast})

In [13]:
only_water_temperature=data['Mean Watertemperature per Hour'].round(decimals = 1).to_frame(name='Mean Watertemperature per Hour')

In [14]:
df= only_water_temperature.copy()
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled  = min_max_scaler.fit_transform(np.reshape(df['Mean Watertemperature per Hour'].values, (df.shape[0], 1)))
df_scaled  = pd.DataFrame(data=df_scaled , columns=['Mean Watertemperature per Hour'])
n_in = 15 #Timeseries consists of 90 input steps.
n_out = 1
sdf = timeseries_to_supervised(df_scaled, n_in, n_out)
X, y = sdf[[('Mean Watertemperature per Hour(t-%d)' % i) for i in range(15, 0, -1)]].values, sdf['Mean Watertemperature per Hour(t)'].values
len_data = X.shape[0]
print("Data size: %d" % len_data)
train_size = int(len_data * .7)# 70% for training and 30% for testing
print ("Train  data size: %d" % train_size)
print ("Test data size: %d" % (len_data - train_size))

xtr, ytr = X[:train_size, :], y[:train_size]
xte, yte = X[train_size:, :], y[train_size:]
test_dates=data[train_size:].iloc[15:].Date #Keep the dates of the test dataset for display purposes
print(xtr.shape, ytr.shape)
print(xte.shape, yte.shape)

Data size: 8650
Train  data size: 6055
Test data size: 2595
(6055, 15) (6055,)
(2595, 15) (2595,)


In [44]:
batch_size = 1 #1
model1 = Sequential()

model1.add(Dense(units=50, input_dim=xtr.shape[1], activation="relu"))#input dim is 4 as the timeseries steps.
model1.add(Dense(50, activation="relu"))# linear activation functios was used as well
model1.add(Dense(1))
model1.compile(loss='mean_squared_error', optimizer='adam')
model1.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 50)                800       
_________________________________________________________________
dense_13 (Dense)             (None, 50)                2550      
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 51        
Total params: 3,401
Trainable params: 3,401
Non-trainable params: 0
_________________________________________________________________


In [45]:
model1.fit(xtr,ytr, epochs=10, batch_size=batch_size, verbose=2)

Epoch 1/10
6055/6055 - 2s - loss: 5.0185e-04
Epoch 2/10
6055/6055 - 2s - loss: 3.1159e-04
Epoch 3/10
6055/6055 - 2s - loss: 2.8112e-04
Epoch 4/10
6055/6055 - 2s - loss: 2.4815e-04
Epoch 5/10
6055/6055 - 2s - loss: 2.4579e-04
Epoch 6/10
6055/6055 - 2s - loss: 2.2688e-04
Epoch 7/10
6055/6055 - 2s - loss: 2.1629e-04
Epoch 8/10
6055/6055 - 2s - loss: 2.2946e-04
Epoch 9/10
6055/6055 - 2s - loss: 2.0540e-04
Epoch 10/10
6055/6055 - 2s - loss: 1.9720e-04


In [46]:
testPredict = model1.predict(xte, batch_size=batch_size)

In [47]:
yte2d = np.reshape(yte, (yte.shape[0], 1)) 
testPredict = min_max_scaler.inverse_transform(testPredict)
testY = min_max_scaler.inverse_transform(yte2d)
print("Test MSE: ", mean_squared_error(testY, testPredict))
print("Test MAE: ", sum(abs(testY-testPredict))/testY.shape[0])
print("Test R2: ", r2_score(testY, testPredict))
forecast=df_scaled[-15:]
forecast=forecast.to_numpy()
forecast=forecast.reshape(1,15)
n_past = 15
n_days_for_testing_prediction=16  #predict the next hour
predict_period_dates = pd.date_range(list(test_dates)[-n_past], periods=n_days_for_testing_prediction,freq='h').tolist()
predict_period_dates=predict_period_dates[-1:]

Test MSE:  0.436794805803773
Test MAE:  [0.47646218]
Test R2:  0.9006157383404192


In [19]:
forecast = model1.predict(forecast, batch_size=batch_size)#predict

In [20]:
forecast = min_max_scaler.inverse_transform(forecast)#inverse
forecast=np.squeeze(forecast)#reshape
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i)

In [21]:
df_forecast_water_temperature = pd.DataFrame({'Date':np.array(forecast_dates), 'Mean Watertemperature per Hour Forecasted':forecast})

In [22]:
data

,Date,Atm.Temperature,Mean Watertemperature per Hour,Mean Salinity per Hour,Mean Conductivity per Hour,Mean Chlorofyl per Hour
0,2021-04-06 11:00:00,4.9,9.900000,0.473333,680.166667,4.466667
1,2021-04-06 12:00:00,5.5,9.900000,0.470000,676.666667,5.000000
2,2021-04-06 13:00:00,4.5,9.916667,0.475000,681.833333,6.066667
3,2021-04-06 14:00:00,1.8,9.891667,0.475000,682.000000,7.108333
4,2021-04-06 15:00:00,2.2,9.800000,0.477500,682.750000,7.375000
...,...,...,...,...,...,...
8660,2022-04-02 20:00:00,1.4,10.200000,0.500000,723.166667,10.233333
8661,2022-04-02 21:00:00,1.2,10.125000,0.498333,717.416667,9.558333
8662,2022-04-02 22:00:00,0.9,10.100000,0.505833,726.833333,9.908333
8663,2022-04-02 23:00:00,-0.2,10.075000,0.510000,734.833333,9.791667


In [23]:
only_salinity=data['Mean Salinity per Hour'].round(decimals = 1).to_frame(name='Mean Salinity per Hour')

In [24]:
df= only_salinity.copy()
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled  = min_max_scaler.fit_transform(np.reshape(df['Mean Salinity per Hour'].values, (df.shape[0], 1)))
df_scaled  = pd.DataFrame(data=df_scaled , columns=['Mean Salinity per Hour'])
n_in = 15 #Timeseries consists of 90 input steps.
n_out = 1
sdf = timeseries_to_supervised(df_scaled, n_in, n_out)
X, y = sdf[[('Mean Salinity per Hour(t-%d)' % i) for i in range(15, 0, -1)]].values, sdf['Mean Salinity per Hour(t)'].values
len_data = X.shape[0]
print("Data size: %d" % len_data)
train_size = int(len_data * .7)# 70% for training and 30% for testing
print ("Train  data size: %d" % train_size)
print ("Test data size: %d" % (len_data - train_size))

xtr, ytr = X[:train_size, :], y[:train_size]
xte, yte = X[train_size:, :], y[train_size:]
test_dates=data[train_size:].iloc[15:].Date #Keep the dates of the test dataset for display purposes
print(xtr.shape, ytr.shape)
print(xte.shape, yte.shape)

Data size: 8650
Train  data size: 6055
Test data size: 2595
(6055, 15) (6055,)
(2595, 15) (2595,)


In [25]:
batch_size = 1 #1
model2 = Sequential()

model2.add(Dense(units=50, input_dim=xtr.shape[1], activation="relu"))#input dim is 4 as the timeseries steps.
model2.add(Dense(50, activation="relu"))# linear activation functios was used as well
model2.add(Dense(1))
model2.compile(loss='mean_squared_error', optimizer='adam')
model2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 50)                800       
_________________________________________________________________
dense_7 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 51        
Total params: 3,401
Trainable params: 3,401
Non-trainable params: 0
_________________________________________________________________


In [26]:
model2.fit(xtr,ytr, epochs=10, batch_size=batch_size, verbose=2)

Epoch 1/10
6055/6055 - 2s - loss: 0.0120
Epoch 2/10
6055/6055 - 2s - loss: 0.0085
Epoch 3/10
6055/6055 - 2s - loss: 0.0082
Epoch 4/10
6055/6055 - 2s - loss: 0.0079
Epoch 5/10
6055/6055 - 2s - loss: 0.0079
Epoch 6/10
6055/6055 - 2s - loss: 0.0079
Epoch 7/10
6055/6055 - 2s - loss: 0.0078
Epoch 8/10
6055/6055 - 2s - loss: 0.0076
Epoch 9/10
6055/6055 - 2s - loss: 0.0077
Epoch 10/10
6055/6055 - 2s - loss: 0.0076


In [27]:
testPredict = model2.predict(xte, batch_size=batch_size)

In [28]:
yte2d = np.reshape(yte, (yte.shape[0], 1)) 
testPredict = min_max_scaler.inverse_transform(testPredict)
testY = min_max_scaler.inverse_transform(yte2d)
print("Test MSE: ", mean_squared_error(testY, testPredict))
print("Test MAE: ", sum(abs(testY-testPredict))/testY.shape[0])
print("Test R2: ", r2_score(testY, testPredict))
forecast=df_scaled[-15:]
forecast=forecast.to_numpy()
forecast=forecast.reshape(1,15)
n_past = 15
n_days_for_testing_prediction=16  #predict the next hour
predict_period_dates = pd.date_range(list(test_dates)[-n_past], periods=n_days_for_testing_prediction,freq='h').tolist()
predict_period_dates=predict_period_dates[-1:]

Test MSE:  0.00027556191043417945
Test MAE:  [0.00649373]
Test R2:  0.9273381671455003


In [29]:
forecast = model2.predict(forecast, batch_size=batch_size)#predict

In [30]:
forecast = min_max_scaler.inverse_transform(forecast)#inverse
forecast=np.squeeze(forecast)#reshape
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i)

In [31]:
df_forecast_salinity = pd.DataFrame({'Date':np.array(forecast_dates), 'Salinity Forecasted':forecast})

In [32]:
only_chlorofyl=data['Mean Chlorofyl per Hour'].round(decimals = 2).to_frame(name='Mean Chlorofyl per Hour')

In [33]:
df= only_chlorofyl.copy()
min_max_scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled  = min_max_scaler.fit_transform(np.reshape(df['Mean Chlorofyl per Hour'].values, (df.shape[0], 1)))
df_scaled  = pd.DataFrame(data=df_scaled , columns=['Mean Chlorofyl per Hour'])
n_in = 15 #Timeseries consists of 90 input steps.
n_out = 1
sdf = timeseries_to_supervised(df_scaled, n_in, n_out)
X, y = sdf[[('Mean Chlorofyl per Hour(t-%d)' % i) for i in range(15, 0, -1)]].values, sdf['Mean Chlorofyl per Hour(t)'].values
len_data = X.shape[0]
print("Data size: %d" % len_data)
train_size = int(len_data * .7)# 70% for training and 30% for testing
print ("Train  data size: %d" % train_size)
print ("Test data size: %d" % (len_data - train_size))

xtr, ytr = X[:train_size, :], y[:train_size]
xte, yte = X[train_size:, :], y[train_size:]
test_dates=data[train_size:].iloc[15:].Date #Keep the dates of the test dataset for display purposes
print(xtr.shape, ytr.shape)
print(xte.shape, yte.shape)

Data size: 8650
Train  data size: 6055
Test data size: 2595
(6055, 15) (6055,)
(2595, 15) (2595,)


In [34]:
batch_size = 1 #1
model3 = Sequential()

model3.add(Dense(units=50, input_dim=xtr.shape[1], activation="relu"))#input dim is 4 as the timeseries steps.
model3.add(Dense(50, activation="relu"))# linear activation functios was used as well
model3.add(Dropout(0.2))
model3.add(Dense(1))
model3.compile(loss='mean_squared_error', optimizer='adam')
model3.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 50)                800       
_________________________________________________________________
dense_10 (Dense)             (None, 50)                2550      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 51        
Total params: 3,401
Trainable params: 3,401
Non-trainable params: 0
_________________________________________________________________


In [51]:
model3.fit(xtr,ytr, epochs=10, batch_size=batch_size, verbose=2)

Epoch 1/10
6055/6055 - 2s - loss: 3.0141e-04
Epoch 2/10
6055/6055 - 2s - loss: 2.9396e-04
Epoch 3/10
6055/6055 - 2s - loss: 2.5294e-04
Epoch 4/10
6055/6055 - 2s - loss: 3.1740e-04
Epoch 5/10
6055/6055 - 2s - loss: 2.4301e-04
Epoch 6/10
6055/6055 - 2s - loss: 2.8037e-04
Epoch 7/10
6055/6055 - 2s - loss: 3.0866e-04
Epoch 8/10
6055/6055 - 2s - loss: 3.0818e-04
Epoch 9/10
6055/6055 - 2s - loss: 2.9615e-04
Epoch 10/10
6055/6055 - 2s - loss: 2.5028e-04


In [52]:
testPredict = model3.predict(xte, batch_size=batch_size)

In [53]:
yte2d = np.reshape(yte, (yte.shape[0], 1)) 
testPredict = min_max_scaler.inverse_transform(testPredict)
testY = min_max_scaler.inverse_transform(yte2d)
print("Test MSE: ", mean_squared_error(testY, testPredict))
print("Test MAE: ", sum(abs(testY-testPredict))/testY.shape[0])
print("Test R2: ", r2_score(testY, testPredict))
forecast=df_scaled[-15:]
forecast=forecast.to_numpy()
forecast=forecast.reshape(1,15)
n_past = 15
n_days_for_testing_prediction=16  #predict the next hour
predict_period_dates = pd.date_range(list(test_dates)[-n_past], periods=n_days_for_testing_prediction,freq='h').tolist()
predict_period_dates=predict_period_dates[-1:]

Test MSE:  0.6747034511334794
Test MAE:  [0.71616062]
Test R2:  0.8464841993560797


In [38]:
forecast = model3.predict(forecast, batch_size=batch_size)#predict

In [39]:
forecast = min_max_scaler.inverse_transform(forecast)#inverse
forecast=np.squeeze(forecast)#reshape
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i)

In [40]:
df_forecast_chlorofyl = pd.DataFrame({'Date':np.array(forecast_dates), 'Chlorofyl Forecasted':forecast})

In [41]:
forecasts=pd.concat((df_forecast_atm_temperature,df_forecast_water_temperature['Mean Watertemperature per Hour Forecasted'],df_forecast_salinity['Salinity Forecasted'],df_forecast_chlorofyl['Chlorofyl Forecasted']),axis=1)

In [42]:
forecasts

,Date,Atm.Temperature Forecasted,Mean Watertemperature per Hour Forecasted,Salinity Forecasted,Chlorofyl Forecasted
0,2022-04-03 01:00:00,0.864817,10.0221,0.495026,11.253048


In [43]:
#forecasts.to_excel(r'forecasted_by_mlps.xlsx',header=True, index=False)

In [55]:
model0.save('atm_temp')
model1.save('water_temp')
model2.save('salinity')
model3.save('chlorofyl')

INFO:tensorflow:Assets written to: atm_temp\assets
INFO:tensorflow:Assets written to: water_temp\assets
INFO:tensorflow:Assets written to: salinity\assets
INFO:tensorflow:Assets written to: chlorofyl\assets


In [58]:
forecasts=forecasts.rename(columns={'Atm.Temperature Forecasted':'Atm.Temperature','Mean Watertemperature per Hour Forecasted':'Mean Watertemperature per Hour','Salinity Forecasted':'Mean Salinity per Hour','Chlorofyl Forecasted':'Mean Chlorofyl per Hour'})
                          

In [67]:
new_df=pd.concat((data[-14:],forecasts),axis=0)

In [68]:
new_df=new_df.reset_index()

In [70]:
new_df

,index,Date,Atm.Temperature,Mean Watertemperature per Hour,Mean Salinity per Hour,Mean Conductivity per Hour,Mean Chlorofyl per Hour
0,8651,2022-04-02 11:00:00,6.700000,10.100000,0.500000,717.750000,9.458333
1,8652,2022-04-02 12:00:00,7.300000,10.175000,0.500833,725.250000,9.641667
2,8653,2022-04-02 13:00:00,7.300000,10.258333,0.505000,731.833333,10.750000
3,8654,2022-04-02 14:00:00,6.600000,10.200000,0.506667,729.166667,8.658333
4,8655,2022-04-02 15:00:00,6.400000,10.200000,0.500833,724.083333,9.841667
5,8656,2022-04-02 16:00:00,6.300000,10.200000,0.507500,733.666667,9.725000
6,8657,2022-04-02 17:00:00,5.200000,10.191667,0.515000,741.000000,11.033333
7,8658,2022-04-02 18:00:00,4.300000,10.116667,0.506667,729.916667,10.741667
8,8659,2022-04-02 19:00:00,3.100000,10.166667,0.509167,731.916667,9.608333
9,8660,2022-04-02 20:00:00,1.400000,10.200000,0.500000,723.166667,10.233333


In [135]:
atm_temp=new_df[['Atm.Temperature']]

In [142]:
water_temp=new_df[['Mean Watertemperature per Hour']]

In [137]:
salinity=new_df[['Mean Salinity per Hour']]

In [138]:
chlorofyl=new_df[['Mean Chlorofyl per Hour']]

In [139]:
i=0
while i<14:
    forecasts=atm_temp.iloc[-15:]
    forecasts_scaled=min_max_scaler.fit_transform(np.reshape(forecasts['Atm.Temperature'].values, (15, 1)))
    forecasts=forecasts_scaled.reshape(1,15)
    forecasts = model0.predict(forecasts, batch_size=batch_size)#predict
    forecasts=min_max_scaler.inverse_transform(forecasts)
    forecasts=pd.DataFrame(data=forecasts,columns=['Atm.Temperature'])
    atm_temp=pd.concat((atm_temp,forecasts),axis=0)
    i=i+1

In [143]:
i=0
while i<14:
    forecasts=water_temp.iloc[-15:]
    forecasts_scaled=min_max_scaler.fit_transform(np.reshape(forecasts['Mean Watertemperature per Hour'].values, (15, 1)))
    #forecasts=forecasts_scaled.to_numpy()
    forecasts=forecasts_scaled.reshape(1,15)
    forecasts = model1.predict(forecasts, batch_size=batch_size)#predict
    forecasts=min_max_scaler.inverse_transform(forecasts)
    forecasts=pd.DataFrame(data=forecasts,columns=['Mean Watertemperature per Hour'])
    water_temp=pd.concat((water_temp,forecasts),axis=0)
    i=i+1

In [145]:
i=0
while i<14:
    forecasts=salinity.iloc[-15:]
    forecasts_scaled=min_max_scaler.fit_transform(np.reshape(forecasts['Mean Salinity per Hour'].values, (15, 1)))
    #forecasts=forecasts_scaled.to_numpy()
    forecasts=forecasts_scaled.reshape(1,15)
    forecasts = model2.predict(forecasts, batch_size=batch_size)#predict
    forecasts=min_max_scaler.inverse_transform(forecasts)
    forecasts=pd.DataFrame(data=forecasts,columns=['Mean Salinity per Hour'])
    salinity=pd.concat((salinity,forecasts),axis=0)
    i=i+1

In [147]:
i=0
while i<14:
    forecasts=chlorofyl.iloc[-15:]
    forecasts_scaled=min_max_scaler.fit_transform(np.reshape(forecasts['Mean Chlorofyl per Hour'].values, (15, 1)))
    #forecasts=forecasts_scaled.to_numpy()
    forecasts=forecasts_scaled.reshape(1,15)
    forecasts = model3.predict(forecasts, batch_size=batch_size)#predict
    forecasts=min_max_scaler.inverse_transform(forecasts)
    forecasts=pd.DataFrame(data=forecasts,columns=['Mean Chlorofyl per Hour'])
    chlorofyl=pd.concat((chlorofyl,forecasts),axis=0)
    i=i+1

In [149]:
atm_temp=atm_temp.reset_index()

In [150]:
water_temp=water_temp.reset_index()

In [151]:
salinity=salinity.reset_index()

In [152]:
chlorofyl=chlorofyl.reset_index()

In [155]:
final=pd.concat((atm_temp,water_temp,salinity,chlorofyl),axis=1)

In [157]:
final=final.drop(columns={'index'})

In [159]:
final=final.iloc[-14:]

In [160]:
final

,Atm.Temperature,Mean Watertemperature per Hour,Mean Salinity per Hour,Mean Chlorofyl per Hour
15,1.123779,10.033001,0.502815,11.150275
16,1.635482,10.028564,0.503048,10.831940
17,2.138935,10.036883,0.505099,10.777822
18,2.702619,10.044598,0.505340,10.390002
19,3.545437,10.048436,0.506910,10.141741
20,4.232475,10.047326,0.504868,9.852780
21,4.705771,10.048590,0.505936,9.729282
22,5.024044,10.036609,0.504162,9.704208
23,4.953002,10.017195,0.505593,9.606123
24,4.698179,10.010398,0.506449,9.667429


In [179]:
n_past =14
n_days_for_testing_prediction=28  #predict the next hour
predict_period_dates = pd.date_range(list(test_dates)[-n_past], periods=n_days_for_testing_prediction,freq='h').tolist()
predict_period_dates=predict_period_dates[-14:]

In [181]:
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i)

In [183]:
forecast_dates = pd.DataFrame({'Date':np.array(forecast_dates)})

In [186]:
final=final.reset_index(drop=True)

In [187]:
final

,Atm.Temperature,Mean Watertemperature per Hour,Mean Salinity per Hour,Mean Chlorofyl per Hour
0,1.123779,10.033001,0.502815,11.150275
1,1.635482,10.028564,0.503048,10.831940
2,2.138935,10.036883,0.505099,10.777822
3,2.702619,10.044598,0.505340,10.390002
4,3.545437,10.048436,0.506910,10.141741
5,4.232475,10.047326,0.504868,9.852780
6,4.705771,10.048590,0.505936,9.729282
7,5.024044,10.036609,0.504162,9.704208
8,4.953002,10.017195,0.505593,9.606123
9,4.698179,10.010398,0.506449,9.667429


In [188]:
final=pd.concat((final,forecast_dates),axis=1)

In [189]:
final

,Atm.Temperature,Mean Watertemperature per Hour,Mean Salinity per Hour,Mean Chlorofyl per Hour,Date
0,1.123779,10.033001,0.502815,11.150275,2022-04-03 01:00:00
1,1.635482,10.028564,0.503048,10.831940,2022-04-03 02:00:00
2,2.138935,10.036883,0.505099,10.777822,2022-04-03 03:00:00
3,2.702619,10.044598,0.505340,10.390002,2022-04-03 04:00:00
4,3.545437,10.048436,0.506910,10.141741,2022-04-03 05:00:00
5,4.232475,10.047326,0.504868,9.852780,2022-04-03 06:00:00
6,4.705771,10.048590,0.505936,9.729282,2022-04-03 07:00:00
7,5.024044,10.036609,0.504162,9.704208,2022-04-03 08:00:00
8,4.953002,10.017195,0.505593,9.606123,2022-04-03 09:00:00
9,4.698179,10.010398,0.506449,9.667429,2022-04-03 10:00:00


In [190]:
final.to_excel(r'forecasted_by_mlps.xlsx',header=True, index=False)